# Метод внутренней точки
### Замятина Екатерина 476 группа

In [57]:
import pandas as pd
import math

In [15]:
data = pd.read_csv('data1.csv', encoding='utf7').astype(float)

In [16]:
r = data.median(0)

### Среднее значение прибыли

In [17]:
print r

SBER       0.003859
MEGAFON   -0.002380
YANDEX    -0.000475
ROSN      -0.003660
dtype: float64


In [18]:
R = 0.012
sigma = 0.1

### Матрица ковариации

In [19]:
data.cov()

,SBER,MEGAFON,YANDEX,ROSN
SBER,0.001747,0.000317,0.001005,0.001107
MEGAFON,0.000317,0.001488,0.000438,0.000527
YANDEX,0.001005,0.000438,0.002834,0.000879
ROSN,0.001107,0.000527,0.000879,0.001927


In [20]:
import numpy as np

In [104]:
omega = np.array(data.cov())

In [46]:
def f(x):
    x = np.array(x)
    return 0.5*np.dot(np.dot(x.T, omega), x)

In [60]:
def g(x):
    g = 0
    for i in range(4):
       g = g + math.log(1 - x[i]) 
    return g

### Гессиан

$\nabla ^2 = \Omega- p(\dfrac{1}{(1 - \sum x_i)^2} + \dfrac{(r,r)}{(R - rx)^2})*
\begin{pmatrix}
1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 \\        
1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1
\end{pmatrix}$

In [123]:
def gess_g(x,p):
    return omega - p* ((1 - np.sum(x))**(-2) + np.dot(r,r)*(R - np.dot(r,x))**(-2))*np.ones((4,4))

### Градиен

Градиент:
$\nabla g(x_k) = \Omega x - p(\dfrac{\bf{1}}{1 - \sum x_i} + \dfrac{r}{R - xr})$

In [119]:
def grad_g(x,p):
    return np.dot(omega,x) - p*x/(1 - np.sum(x))

### Метод Ньютона

$argmin(\frac{1}{2}(x^T \Sigma x) - p\sum log(-h_i(x))) $

$ h(x) = 1 - \sum x_i$

$ h(x) = R - \sum x_i*r_i $

Метод Ньютона:

$x_{k+1} = x_k - [\nabla^2 g(x_k)]^{-1} \nabla g(x_k)$

$ g(x_k) = \frac{1}{2}(x^T \Sigma x) - p\sum log(-h_i(x))$

In [120]:
def argmin(x, p, max_number_iterations = 500000, eps = 10**(-8)):
    current_x = x
    current_f = f(current_x)
    function = f(x) - p*(g(x) + math.log(math.fabs(np.dot(x.T, r) - R)))
    for i in range(max_number_iterations):
        if (i > 0):
            if np.dot(current_x - previous_x, current_x - previous_x) < eps*eps:
                return (current_x, "ok")
        previous_x = current_x
        current_x = previous_x - np.dot(np.linalg.inv(gess_g(x,p)), grad_g(x,p))
        current_p = sigma * p
    return (current_x, "error")                 
    

### Метод внутренней точки

$x_{k+1} = argmin(\frac{1}{2}(x^T \Sigma x) - p\sum log(-h_i(x))) $

In [121]:
def method(p, x, max_number_iterations, eps = 10**(-8)):
    current_x = x
    current_p = p
    for i in range(max_number_iterations):
        if (i > 0):
            if np.dot(current_x - previous_x, current_x - previous_x) < eps*eps:
                return (current_x, f(x), i)
        previous_x = current_x
        current_x, message = argmin(current_x, p)
        if (message == "error"):
            print "Метод Ньютона не сошелся"
            return
        current_p = sigma * p
    return (current_x, f(x), "over 500000")

In [131]:
start_point = 0.2*np.ones(4)
df2 = pd.DataFrame({ 'p' : method(5,start_point, max_number_iterations=5000), 
                    }, index = ["x", "f(x)", "method converged at iteration"])

Метод Ньютона не сошелся
